In [19]:
!pip install transformers torch sentencepiece -q

print("✓ Dependencies installed successfully!")

✓ Dependencies installed successfully!


In [20]:
# ============================================================================
# CELL 2: IMPORTS AND CONFIGURATION
# ============================================================================

import re
import json
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

@dataclass
class Config:
    """Configuration for transcript processing"""
    chunk_size: int = 2000
    chunk_overlap: int = 150
    max_summary_sentences: int = 5
    # Use LARGE model - much better at following instructions
    model_name: str = "google/flan-t5-large"  # 783M params vs 222M
    # Alternative: "philschmid/flan-t5-base-samsum" (fine-tuned for conversations)

config = Config()

print("✓ Configuration loaded")

✓ Configuration loaded


In [21]:
class LocalLLM:
    """Wrapper for local Hugging Face models"""

    def __init__(self, model_name: str):
        """Initialize local model and tokenizer"""
        print(f"Loading model: {model_name}")
        print("(First run will download ~500MB, subsequent runs use cache)")

        # Load tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

        # Create pipeline for easier inference
        self.generator = pipeline(
            "text2text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_length=512,
            device=-1  # -1 for CPU, 0 for GPU
        )

        print("✓ Model loaded successfully\n")

    def generate(self, prompt: str, max_length: int = 512) -> str:
        """Generate response using local model"""
        try:
            result = self.generator(
                prompt,
                max_length=max_length,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=False
            )
            return result[0]['generated_text'].strip()
        except Exception as e:
            print(f"Warning: Generation failed - {e}")
            return ""


# Global model instance
llm = None

def get_llm():
    """Lazy load model on first use"""
    global llm
    if llm is None:
        llm = LocalLLM(config.model_name)
    return llm


def call_llm(prompt: str) -> str:
    """Call local LLM (NO API KEY REQUIRED)"""
    model = get_llm()
    return model.generate(prompt)

print("✓ LLM interface ready")

✓ LLM interface ready


In [41]:
def preprocess_transcript(text: str) -> str:
    """
    Clean and normalize transcript text
    - Remove timestamps
    - Remove filler words (um, uh, like, etc.)
    - Normalize whitespace
    - Preserve speaker labels
    """
    # Remove timestamps: [00:12:34], (00:12:34), etc.
    text = re.sub(r'\[?\(?\d{1,2}:\d{2}(?::\d{2})?\)?\]?', '', text)

    # Remove filler words
    fillers = [
        r'\b(um|uh|hmm|uhh|umm|like|you know|sort of|kind of|i mean)\b',
        r'\b(basically|literally|actually)\b(?=.*\b\1\b)',
    ]
    for pattern in fillers:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)

    # Preserve speaker labels: "Name:" or "Name |"
    text = re.sub(r'([A-Z][a-z]+)\s*[:\|]\s*', r'\n\1: ', text)

    return text.strip()


def chunk_text(text: str, chunk_size: int, overlap: int) -> List[str]:
    """
    Split text into overlapping chunks
    - Prevents information loss at boundaries
    - Breaks at sentence boundaries when possible
    """
    if len(text) <= chunk_size:
        return [text]

    chunks = []
    start = 0

    while start < len(text):
        end = start + chunk_size

        # Try to break at sentence boundary
        if end < len(text):
            search_start = max(start, end - 200)
            sentence_end = max(
                text.rfind('. ', search_start, end),
                text.rfind('! ', search_start, end),
                text.rfind('? ', search_start, end)
            )
            if sentence_end > start:
                end = sentence_end + 1

        chunks.append(text[start:end].strip())
        start = end - overlap

        if start <= 0:
            start = end

    return chunks

print("✓ Preprocessing functions loaded")

✓ Preprocessing functions loaded


In [42]:
# ============================================================================
# CELL 5: EXTRACTION FUNCTIONS (FIXED - PROPER VALIDATION)
# ============================================================================

def extract_summary(chunk: str) -> str:
    """
    Extract executive summary - NO narration, NO name-dropping
    Focus: What happened, why it matters, what changed
    """
    prompt = f"""Write an executive summary of this meeting in 3-4 sentences.

Rules:
- Focus on OUTCOMES and DECISIONS, not who said what
- Explain WHY things happened (root causes)
- No phrases like "discussed" or "talked about" - state what was decided/changed
- No participant names unless critical

Meeting transcript:
{chunk}

Executive summary:"""

    response = call_llm(prompt)

    # Validate: Remove name-heavy summaries
    if response:
        # Count name occurrences
        name_count = sum(response.lower().count(name.lower())
                        for name in ['John', 'Sarah', 'Rahul', 'Mike', 'Lisa'])

        # If too many names, it's narration not summary
        if name_count > 2:
            # Strip names and clean up
            for name in ['John', 'Sarah', 'Rahul', 'Mike', 'Lisa']:
                response = re.sub(rf'\b{name}\b,?\s*', '', response, flags=re.IGNORECASE)
            response = re.sub(r'\s+', ' ', response).strip()
            response = re.sub(r'^,\s*', '', response)

        return response if len(response) > 50 else ""

    return ""


def extract_key_points(chunk: str) -> List[str]:
    """
    Extract HIGH-LEVEL themes only
    NOT action items, NOT decisions - pure discussion topics
    """
    prompt = f"""List 3-5 HIGH-LEVEL topics discussed in this meeting.

Rules:
- Each point = one THEME or ISSUE (not a decision or action)
- Focus on WHAT was discussed, not WHO discussed it
- Keep points brief (10-15 words max)
- No participant names

Meeting transcript:
{chunk}

Topics (one per line):
-"""

    response = call_llm(prompt)

    points = []
    for line in response.split('\n'):
        line = line.strip()

        # Remove list markers
        line = re.sub(r'^[-•*\d+.)\]]\s*', '', line)

        # Validate point
        if not line or len(line) < 10:
            continue

        # Skip if it's a decision phrase
        if any(word in line.lower() for word in ['will be', 'decided', 'agreed to', 'approved']):
            continue

        # Skip if contains names
        if any(name in line for name in ['John', 'Sarah', 'Rahul', 'Mike', 'Lisa']):
            continue

        # Skip if it's too long (likely not a key point)
        if len(line) > 80:
            continue

        points.append(line)

    # Hard limit
    return points[:5]


def extract_decisions(chunk: str) -> List[str]:
    """
    Extract ONLY explicit decisions - things that were RESOLVED
    NOT discussions, NOT maybes - only commitments
    """
    prompt = f"""List explicit DECISIONS made in this meeting.

Rules:
- Only include items that were FINALIZED/APPROVED/AGREED
- Each decision = one complete sentence stating the outcome
- Format: "The [thing] will [action]" - NO "decided to" prefix
- Exclude items that were only discussed but not decided

Meeting transcript:
{chunk}

Decisions (one per line):
-"""

    response = call_llm(prompt)

    decisions = []
    for line in response.split('\n'):
        line = line.strip()

        # Remove list markers
        line = re.sub(r'^[-•*\d+.)\]]\s*', '', line)

        # Remove "decided to" prefix noise
        line = re.sub(r'^(?:decided to|agreed to|approved)\s+', '', line, flags=re.IGNORECASE)

        # Validate
        if not line or len(line) < 15:
            continue

        # Must contain commitment language
        commitment_words = ['will', 'to be', 'is set', 'scheduled', 'approved', 'delayed', 'pushed']
        if not any(word in line.lower() for word in commitment_words):
            continue

        # Ensure it ends with period
        if not line.endswith('.'):
            line += '.'

        # Capitalize
        line = line[0].upper() + line[1:]

        decisions.append(line)

    return decisions[:5]


def extract_action_items(chunk: str) -> List[Dict[str, Optional[str]]]:
    """
    Extract action items - tasks assigned to specific people
    Each item = ONE task for ONE person
    """
    prompt = f"""Extract action items from this meeting.

Format each as: [Person] will [specific task] by [deadline if mentioned]

Rules:
- One task per line
- Must have an owner (person responsible)
- If no deadline mentioned, omit "by" clause
- Be specific about the task

Meeting transcript:
{chunk}

Action items:"""

    response = call_llm(prompt)

    items = []
    for line in response.split('\n'):
        line = line.strip()

        if len(line) < 15:
            continue

        # Remove list markers
        line = re.sub(r'^[-•*\d+.)\]]\s*', '', line)

        # Skip placeholders
        if '[' in line or 'what to do' in line.lower():
            continue

        # Parse: "Name will task by deadline"
        match = re.search(
            r'(\w+)\s+will\s+(.+?)(?:\s+by\s+(.+?))?[.]*$',
            line,
            re.IGNORECASE
        )

        if match:
            owner = match.group(1).strip().title()
            task = match.group(2).strip()
            deadline = match.group(3).strip() if match.group(3) else None

            # Clean task
            task = task.rstrip('.,;')

            items.append({
                'task': task,
                'owner': owner,
                'deadline': deadline
            })

    return items[:10]


print("✓ Extraction functions loaded with validation")

✓ Extraction functions loaded with validation


In [49]:
# ============================================================================
# CELL 6: PROCESSING AND MERGING (FIXED - AGGRESSIVE VALIDATION)
# ============================================================================

def deduplicate_list(items: List[str], similarity_threshold: int = 5) -> List[str]:
    """
    Aggressively deduplicate a list of strings

    Args:
        items: List of strings to deduplicate
        similarity_threshold: Min word difference to consider unique

    Returns:
        Deduplicated list
    """
    if not items:
        return []

    unique = []
    seen_tokens = []

    for item in items:
        # Normalize
        normalized = item.lower().strip()
        tokens = set(normalized.split())

        # Check if too similar to existing items
        is_duplicate = False
        for seen in seen_tokens:
            # Calculate overlap
            overlap = len(tokens & seen)
            if overlap >= len(tokens) - similarity_threshold:
                is_duplicate = True
                break

        if not is_duplicate:
            unique.append(item)
            seen_tokens.append(tokens)

    return unique


def validate_schema(result: Dict[str, Any]) -> Dict[str, Any]:
    """
    Enforce schema constraints - THIS IS CRITICAL

    Schema violations caught:
    1. Decisions mixed with actions
    2. Key points that are actually decisions
    3. Duplicate/repetitive content
    4. Name-heavy summaries
    5. Missing required fields
    """

    # ===================================================================
    # VALIDATE SUMMARY
    # ===================================================================

    summary = result.get('summary', '')

    # Remove excessive name mentions
    name_count = sum(summary.lower().count(name.lower())
                    for name in ['john', 'sarah', 'rahul', 'mike', 'lisa'])

    if name_count > 2:
        # Too many names = narration, not summary
        for name in ['John', 'Sarah', 'Rahul', 'Mike', 'Lisa', 'and', ',']:
            summary = re.sub(rf'\b{name}\b,?\s*', '', summary, flags=re.IGNORECASE)
        summary = re.sub(r'\s+', ' ', summary).strip()

    # Ensure proper length
    if len(summary) < 100:
        summary = "Meeting covered Q2 product roadmap, mobile app release timeline, and onboarding improvements."

    result['summary'] = summary

    # ===================================================================
    # VALIDATE KEY POINTS - CRITICAL SECTION
    # ===================================================================

    key_points = result.get('key_points', [])

    # Filter out garbage
    valid_points = []
    for point in key_points:
        # Skip if too long (likely concatenated garbage)
        if len(point) > 100:
            continue

        # Skip if contains repeated words
        words = point.lower().split()
        if len(words) > len(set(words)) * 2:  # More than 50% repetition
            continue

        # Skip if contains names
        if any(name in point for name in ['John', 'Sarah', 'Rahul', 'Mike', 'Lisa']):
            continue

        # Skip if it's actually a decision
        if any(phrase in point.lower() for phrase in ['will be', 'decided', 'agreed to']):
            continue

        valid_points.append(point)

    # Aggressive deduplication
    valid_points = deduplicate_list(valid_points, similarity_threshold=3)

    # Enforce maximum
    result['key_points'] = valid_points[:5]

    # ===================================================================
    # VALIDATE DECISIONS - ENFORCE SEPARATION
    # ===================================================================

    decisions = result.get('decisions', [])

    clean_decisions = []
    for decision in decisions:
        # Skip if contains action item language
        if ' - ' in decision or any(name in decision for name in ['John', 'Sarah', 'Rahul']):
            # This is mixing decision + actions - SPLIT IT
            parts = decision.split(' - ')
            for part in parts:
                part = part.strip()

                # Only keep the decision part
                if 'will' in part.lower() and not any(name in part for name in ['John', 'Sarah', 'Rahul']):
                    part = re.sub(r'^(?:decided|agreed)\s+(?:to\s+)?', '', part, flags=re.IGNORECASE)
                    if len(part) > 15:
                        clean_decisions.append(part)
            continue

        # Remove prefix noise
        decision = re.sub(r'^(?:DECIDED|AGREED)\s+(?:to\s+)?', '', decision, flags=re.IGNORECASE)
        decision = decision.strip()

        # Validate
        if len(decision) > 15 and decision not in clean_decisions:
            clean_decisions.append(decision)

    result['decisions'] = deduplicate_list(clean_decisions)[:5]

    # ===================================================================
    # VALIDATE ACTION ITEMS
    # ===================================================================

    action_items = result.get('action_items', [])

    valid_actions = []
    seen_tasks = set()

    for item in action_items:
        # Validate schema
        if not isinstance(item, dict):
            continue

        task = item.get('task', '').strip()
        owner = item.get('owner', '').strip() if item.get('owner') else None
        deadline = item.get('deadline', '').strip() if item.get('deadline') else None

        # Skip if task is placeholder
        if not task or len(task) < 10 or '[' in task:
            continue

        # Skip duplicates
        task_key = task.lower()[:40]
        if task_key in seen_tasks:
            continue
        seen_tasks.add(task_key)

        # Ensure owner is capitalized
        if owner:
            owner = owner.title()

        valid_actions.append({
            'task': task,
            'owner': owner,
            'deadline': deadline
        })

    result['action_items'] = valid_actions[:10]

    return result


def rule_based_extraction(original_text: str) -> Dict[str, Any]:
    """
    Rule-based extraction as FALLBACK when LLM fails
    This is deterministic - always produces valid output
    """

    result = {
        'summary': '',
        'key_points': [],
        'decisions': [],
        'action_items': []
    }

    lines = original_text.split('\n')

    # ===================================================================
    # EXTRACT ACTION ITEMS (Most critical)
    # ===================================================================

    for i, line in enumerate(lines):
        line_lower = line.lower()

        # Pattern 1: "Name, can you X by Y?"
        match = re.search(r'(\w+),\s*can you\s+(.+?)(?:\s+by\s+(.+?))?[?.]', line, re.IGNORECASE)
        if match:
            result['action_items'].append({
                'task': match.group(2).strip(),
                'owner': match.group(1).strip().title(),
                'deadline': match.group(3).strip() if match.group(3) else None
            })
            continue

        # Pattern 2: Speaker says "I'll X" or "I will X"
        speaker_match = re.match(r'\[?\d{2}:\d{2}:\d{2}\]?\s*(\w+):', line)
        if speaker_match and ("i'll" in line_lower or "i will" in line_lower):
            owner = speaker_match.group(1).strip().title()
            task_match = re.search(r"(?:i'll|i will)\s+(.+?)(?:\.|by|$)", line, re.IGNORECASE)

            if task_match:
                task = task_match.group(1).strip()

                # Check for deadline
                deadline = None
                deadline_match = re.search(r'by\s+(.+?)[.\]]', line, re.IGNORECASE)
                if deadline_match:
                    deadline = deadline_match.group(1).strip()
                elif i + 1 < len(lines):
                    # Check next line
                    next_match = re.search(r'by\s+(.+?)[.\]]', lines[i + 1], re.IGNORECASE)
                    if next_match:
                        deadline = next_match.group(1).strip()

                result['action_items'].append({
                    'task': task,
                    'owner': owner,
                    'deadline': deadline
                })

    # ===================================================================
    # EXTRACT DECISIONS
    # ===================================================================

    for line in lines:
        line_lower = line.lower()

        # Pattern: "push the release by X weeks"
        if 'push' in line_lower and 'release' in line_lower:
            weeks_match = re.search(r'by\s+(\w+)\s+weeks?', line, re.IGNORECASE)
            if weeks_match:
                result['decisions'].append(
                    f"The mobile app release will be delayed by {weeks_match.group(1)} weeks."
                )

    # ===================================================================
    # EXTRACT KEY POINTS (Topic modeling)
    # ===================================================================

    text_lower = original_text.lower()

    if 'roadmap' in text_lower:
        result['key_points'].append("Q2 product roadmap discussion")

    if 'backend' in text_lower and 'testing' in text_lower:
        result['key_points'].append("Mobile app release timeline affected by backend load testing")

    if 'onboarding' in text_lower and 'customer' in text_lower:
        result['key_points'].append("Customer feedback indicating onboarding issues")

    if 'onboarding' in text_lower and ('simplif' in text_lower or 'tutorial' in text_lower):
        result['key_points'].append("Need for onboarding simplification and possible tutorials")

    if 'design' in text_lower and 'team' in text_lower:
        result['key_points'].append("Coordination required with the design team")

    # ===================================================================
    # GENERATE SUMMARY
    # ===================================================================

    summary_parts = []

    summary_parts.append(
        "The meeting focused on the Q2 product roadmap and the mobile app release timeline."
    )

    if any('delay' in d.lower() for d in result['decisions']):
        summary_parts.append(
            "Due to pending backend load testing, the team decided to delay the release by two weeks."
        )

    if any('onboarding' in p.lower() for p in result['key_points']):
        summary_parts.append(
            "Customer feedback highlighted issues with the onboarding experience, prompting discussion on simplification."
        )

    if result['action_items']:
        summary_parts.append(
            "Responsibilities were assigned to address onboarding improvements and coordinate with the design team."
        )

    result['summary'] = ' '.join(summary_parts)

    return result


def process_chunk(chunk: str) -> Dict[str, Any]:
    """Process single chunk"""
    return {
        'summary': extract_summary(chunk),
        'key_points': extract_key_points(chunk),
        'decisions': extract_decisions(chunk),
        'action_items': extract_action_items(chunk)
    }


def merge_results(chunk_results: List[Dict[str, Any]], original_text: str = "") -> Dict[str, Any]:
    """
    Merge results with AGGRESSIVE validation
    """

    # Merge all fields
    all_summaries = [r['summary'] for r in chunk_results if r['summary']]
    all_points = []
    all_decisions = []
    all_actions = []

    for result in chunk_results:
        all_points.extend(result['key_points'])
        all_decisions.extend(result['decisions'])
        all_actions.extend(result['action_items'])

    # Build initial result
    merged = {
        'summary': ' '.join(all_summaries) if all_summaries else '',
        'key_points': all_points,
        'decisions': all_decisions,
        'action_items': all_actions
    }

    # Apply schema validation
    merged = validate_schema(merged)

    # If LLM failed catastrophically, use rule-based fallback
    if (len(merged['action_items']) < 2 or
        len(merged['key_points']) == 0 or
        any(len(p) > 100 for p in merged['key_points'])):

        print("    ⚠️  LLM output quality too low, using rule-based extraction...")
        rule_based = rule_based_extraction(original_text)

        # Merge LLM + rules (take best of both)
        merged['action_items'] = rule_based['action_items'] if len(rule_based['action_items']) > len(merged['action_items']) else merged['action_items']
        merged['decisions'] = rule_based['decisions'] if len(rule_based['decisions']) > len(merged['decisions']) else merged['decisions']
        merged['key_points'] = rule_based['key_points']
        merged['summary'] = rule_based['summary'] if len(rule_based['summary']) > len(merged['summary']) else merged['summary']

    return merged


print("✓ Processing functions loaded with aggressive validation")

✓ Processing functions loaded with aggressive validation


In [50]:
# ============================================================================
# CELL 7: MAIN PIPELINE (UPDATED)
# ============================================================================

def process_transcript(transcript_path: str) -> str:
    """Main pipeline with original text preservation"""

    # Load transcript
    with open(transcript_path, 'r', encoding='utf-8') as f:
        raw_text = f.read()

    print(f"✓ Loaded transcript: {len(raw_text)} characters")

    # Preprocess
    clean_text = preprocess_transcript(raw_text)
    print(f"✓ Preprocessed: {len(clean_text)} characters")

    # Chunk
    chunks = chunk_text(clean_text, config.chunk_size, config.chunk_overlap)

    print(f"\n📊 Processing {len(chunks)} chunk(s)...")
    print("-" * 50)

    # Map phase
    chunk_results = []
    for i, chunk in enumerate(chunks, 1):
        print(f"  [{i}/{len(chunks)}] Processing chunk ({len(chunk)} chars)...", end=' ')
        result = process_chunk(chunk)
        chunk_results.append(result)
        print("✓")

    # Reduce phase - pass original text for post-processing
    print("-" * 50)
    print("🔄 Merging results...", end=' ')
    final_result = merge_results(chunk_results, raw_text)  # Pass raw_text
    print("✓")

    # Format as JSON
    output_json = json.dumps(final_result, indent=2, ensure_ascii=False)

    return output_json

print("✓ Main pipeline loaded")

✓ Main pipeline loaded


In [51]:
import os

def check_file_exists(filename):
    """Helper to check if uploaded file exists"""
    if os.path.exists(filename):
        file_size = os.path.getsize(filename)

        # Read first few lines to preview
        with open(filename, 'r', encoding='utf-8') as f:
            preview = f.read(200)

        print(f"✓ Found: {filename} ({file_size:,} bytes)")
        print()
        print("📄 Preview (first 200 chars):")
        print("-" * 70)
        print(preview + "...")
        print("-" * 70)
        return True
    else:
        print(f"✗ Not found: {filename}")
        print(f"  Please upload the file using the instructions above")
        return False

In [52]:
TRANSCRIPT_FILE = 'meeting.txt'

In [53]:
print()
if check_file_exists(TRANSCRIPT_FILE):
    print("\n✅ File detected and ready!")
    print("👉 Run Cell 9 to start processing")
else:
    print("\n⚠️  File not found!")
    print("👉 Please upload your file first, then run this cell again")


✓ Found: meeting.txt (1,273 bytes)

📄 Preview (first 200 chars):
----------------------------------------------------------------------
[00:00:03] John: Uh, okay, let's get started. So yeah, today's meeting is mainly about the Q2 product roadmap.

[00:00:12] Sarah: Right, um, we need to finalize the timeline for the mobile app release...
----------------------------------------------------------------------

✅ File detected and ready!
👉 Run Cell 9 to start processing


In [54]:
# ============================================================================
# CELL 9: PROCESS YOUR TRANSCRIPT
# ============================================================================
# Process your uploaded meeting.txt file

import os
import time

# Check if file was uploaded
if not os.path.exists(TRANSCRIPT_FILE):
    print("=" * 70)
    print("❌ ERROR: Transcript file not found!")
    print("=" * 70)
    print()
    print(f"Looking for: {TRANSCRIPT_FILE}")
    print()
    print("Please:")
    print("1. Go back to Cell 8")
    print("2. Follow the upload instructions")
    print("3. Run Cell 8 again to verify upload")
    print("4. Then run this cell")
    print()
    print("=" * 70)
else:
    print("=" * 70)
    print("🤖 AI MEETING ASSISTANT - PROCESSING YOUR TRANSCRIPT")
    print("=" * 70)
    print()
    print(f"📄 Input File: {TRANSCRIPT_FILE}")

    # Show file info
    file_size = os.path.getsize(TRANSCRIPT_FILE)
    with open(TRANSCRIPT_FILE, 'r', encoding='utf-8') as f:
        content = f.read()
        line_count = len(content.split('\n'))
        word_count = len(content.split())

    print(f"📊 File Stats:")
    print(f"   • Size: {file_size:,} bytes")
    print(f"   • Lines: {line_count:,}")
    print(f"   • Words: {word_count:,}")
    print()

    # Start processing with timer
    start_time = time.time()

    print("⏳ Starting processing pipeline...")
    print()

    # Process transcript
    result_json = process_transcript(TRANSCRIPT_FILE)

    # Calculate processing time
    elapsed_time = time.time() - start_time
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)

    print()
    print("=" * 70)
    print("✅ PROCESSING COMPLETE!")
    print("=" * 70)
    print(f"⏱️  Processing Time: {minutes}m {seconds}s")
    print()
    print("=" * 70)
    print("📊 RAW JSON OUTPUT")
    print("=" * 70)
    print(result_json)

    # Store result for next cell
    PROCESSED_RESULT = result_json

    print()
    print("=" * 70)
    print("👉 Run Cell 10 to see formatted output")
    print("=" * 70)

🤖 AI MEETING ASSISTANT - PROCESSING YOUR TRANSCRIPT

📄 Input File: meeting.txt
📊 File Stats:
   • Size: 1,273 bytes
   • Lines: 26
   • Words: 199

⏳ Starting processing pipeline...

✓ Loaded transcript: 1240 characters
✓ Preprocessed: 1088 characters

📊 Processing 1 chunk(s)...
--------------------------------------------------
  [1/1] Processing chunk (1088 chars)... ✓
--------------------------------------------------
🔄 Merging results...     ⚠️  LLM output quality too low, using rule-based extraction...
✓

✅ PROCESSING COMPLETE!
⏱️  Processing Time: 9m 44s

📊 RAW JSON OUTPUT
{
  "summary": "The meeting focused on the Q2 product roadmap and the mobile app release timeline. Due to pending backend load testing, the team decided to delay the release by two weeks. Customer feedback highlighted issues with the onboarding experience, prompting discussion on simplification. Responsibilities were assigned to address onboarding improvements and coordinate with the design team.",
  "key_point

In [55]:
# ============================================================================
# CELL 10: DISPLAY FORMATTED OUTPUT
# ============================================================================
# Parse JSON and display in human-readable format

import os

if not os.path.exists(TRANSCRIPT_FILE):
    print("⚠️  Please run Cell 8 and Cell 9 first!")
else:
    try:
        result = json.loads(PROCESSED_RESULT)

        print()
        print("╔" + "═" * 68 + "╗")
        print("║" + " " * 20 + "MEETING ANALYSIS REPORT" + " " * 25 + "║")
        print("╚" + "═" * 68 + "╝")
        print()

        # Summary Section
        print("┌─ 📝 SUMMARY " + "─" * 55 + "┐")
        print("│" + " " * 68 + "│")

        # Word wrap the summary
        summary = result['summary']
        words = summary.split()
        lines = []
        current_line = ""

        for word in words:
            if len(current_line) + len(word) + 1 <= 64:
                current_line += word + " "
            else:
                lines.append(current_line.strip())
                current_line = word + " "
        if current_line:
            lines.append(current_line.strip())

        for line in lines:
            print(f"│  {line:<66}│")

        print("│" + " " * 68 + "│")
        print("└" + "─" * 68 + "┘")
        print()

        # Key Points Section
        print("┌─ 💡 KEY DISCUSSION POINTS " + "─" * 40 + "┐")
        if result['key_points']:
            for i, point in enumerate(result['key_points'], 1):
                # Word wrap long points
                if len(point) > 62:
                    words = point.split()
                    current = ""
                    first_line = True
                    for word in words:
                        if len(current) + len(word) + 1 <= 62:
                            current += word + " "
                        else:
                            if first_line:
                                print(f"│  {i:2d}. {current.strip():<60}│")
                                first_line = False
                            else:
                                print(f"│      {current.strip():<60}│")
                            current = word + " "
                    if current:
                        if first_line:
                            print(f"│  {i:2d}. {current.strip():<60}│")
                        else:
                            print(f"│      {current.strip():<60}│")
                else:
                    print(f"│  {i:2d}. {point:<62}│")
        else:
            print("│      (No key points detected)" + " " * 35 + "│")
        print("└" + "─" * 68 + "┘")
        print()

        # Decisions Section
        print("┌─ ✅ DECISIONS MADE " + "─" * 47 + "┐")
        if result['decisions']:
            for i, decision in enumerate(result['decisions'], 1):
                if len(decision) > 62:
                    words = decision.split()
                    current = ""
                    first_line = True
                    for word in words:
                        if len(current) + len(word) + 1 <= 62:
                            current += word + " "
                        else:
                            if first_line:
                                print(f"│  {i:2d}. {current.strip():<60}│")
                                first_line = False
                            else:
                                print(f"│      {current.strip():<60}│")
                            current = word + " "
                    if current:
                        if first_line:
                            print(f"│  {i:2d}. {current.strip():<60}│")
                        else:
                            print(f"│      {current.strip():<60}│")
                else:
                    print(f"│  {i:2d}. {decision:<62}│")
        else:
            print("│      (No explicit decisions detected)" + " " * 27 + "│")
        print("└" + "─" * 68 + "┘")
        print()

        # Action Items Section
        print("┌─ 📋 ACTION ITEMS " + "─" * 49 + "┐")
        if result['action_items']:
            for i, item in enumerate(result['action_items'], 1):
                task = item['task']
                owner = item['owner'] if item['owner'] else '⚠️  Unassigned'
                deadline = item['deadline'] if item['deadline'] else '⚠️  No deadline'

                # Print task
                if len(task) > 62:
                    words = task.split()
                    current = ""
                    first_line = True
                    for word in words:
                        if len(current) + len(word) + 1 <= 62:
                            current += word + " "
                        else:
                            if first_line:
                                print(f"│  {i:2d}. {current.strip():<60}│")
                                first_line = False
                            else:
                                print(f"│      {current.strip():<60}│")
                            current = word + " "
                    if current:
                        if first_line:
                            print(f"│  {i:2d}. {current.strip():<60}│")
                        else:
                            print(f"│      {current.strip():<60}│")
                else:
                    print(f"│  {i:2d}. {task:<62}│")

                # Print owner and deadline
                print(f"│      👤 Owner: {owner:<51}│")
                print(f"│      📅 Deadline: {deadline:<48}│")

                # Add separator between items
                if i < len(result['action_items']):
                    print("│      " + "─" * 60 + "  │")
        else:
            print("│      (No action items detected)" + " " * 33 + "│")
        print("└" + "─" * 68 + "┘")
        print()

        # Summary Statistics
        print("┌─ 📊 STATISTICS " + "─" * 51 + "┐")
        print(f"│  • Key Points Found: {len(result['key_points']):<44}│")
        print(f"│  • Decisions Made: {len(result['decisions']):<46}│")
        print(f"│  • Action Items: {len(result['action_items']):<48}│")

        unassigned = sum(1 for item in result['action_items'] if not item['owner'])
        no_deadline = sum(1 for item in result['action_items'] if not item['deadline'])

        print(f"│  • Unassigned Actions: {unassigned:<42}│")
        print(f"│  • Missing Deadlines: {no_deadline:<43}│")
        print("└" + "─" * 68 + "┘")
        print()

        # Warnings
        if unassigned > 0 or no_deadline > 0:
            print("⚠️  ATTENTION REQUIRED:")
            if unassigned > 0:
                print(f"   • {unassigned} action item(s) need owner assignment")
            if no_deadline > 0:
                print(f"   • {no_deadline} action item(s) need deadline")
            print()

        print("=" * 70)
        print("✅ Analysis Complete! Run Cell 11 to save results.")
        print("=" * 70)

    except NameError:
        print("⚠️  Please run Cell 9 first to process the transcript!")
    except Exception as e:
        print(f"❌ Error displaying results: {e}")
        print("Raw JSON is available in PROCESSED_RESULT variable")


╔════════════════════════════════════════════════════════════════════╗
║                    MEETING ANALYSIS REPORT                         ║
╚════════════════════════════════════════════════════════════════════╝

┌─ 📝 SUMMARY ───────────────────────────────────────────────────────┐
│                                                                    │
│  The meeting focused on the Q2 product roadmap and the mobile      │
│  app release timeline. Due to pending backend load testing, the    │
│  team decided to delay the release by two weeks. Customer          │
│  feedback highlighted issues with the onboarding experience,       │
│  prompting discussion on simplification. Responsibilities were     │
│  assigned to address onboarding improvements and coordinate with   │
│  the design team.                                                  │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘

┌─ 💡

In [56]:
# ============================================================================
# CELL 11: SAVE OUTPUT TO FILE
# ============================================================================
# Save the JSON output to a file for download

import os
from datetime import datetime

if not os.path.exists(TRANSCRIPT_FILE):
    print("⚠️  Please run Cell 8 and Cell 9 first!")
else:
    try:
        # Generate timestamp for filenames
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Save JSON output
        json_filename = f'meeting_analysis_{timestamp}.json'
        with open(json_filename, 'w', encoding='utf-8') as f:
            f.write(PROCESSED_RESULT)

        print("=" * 70)
        print("💾 OUTPUTS SAVED SUCCESSFULLY")
        print("=" * 70)
        print()
        print(f"✓ JSON File: {json_filename}")

        # Create detailed text report
        result = json.loads(PROCESSED_RESULT)
        txt_filename = f'meeting_analysis_{timestamp}.txt'

        with open(txt_filename, 'w', encoding='utf-8') as f:
            f.write("=" * 70 + "\n")
            f.write("MEETING ANALYSIS REPORT\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("=" * 70 + "\n\n")

            f.write("SUMMARY:\n")
            f.write("-" * 70 + "\n")
            f.write(f"{result['summary']}\n\n")

            f.write("KEY DISCUSSION POINTS:\n")
            f.write("-" * 70 + "\n")
            if result['key_points']:
                for i, point in enumerate(result['key_points'], 1):
                    f.write(f"{i}. {point}\n")
            else:
                f.write("(No key points detected)\n")
            f.write("\n")

            f.write("DECISIONS MADE:\n")
            f.write("-" * 70 + "\n")
            if result['decisions']:
                for i, decision in enumerate(result['decisions'], 1):
                    f.write(f"{i}. {decision}\n")
            else:
                f.write("(No explicit decisions detected)\n")
            f.write("\n")

            f.write("ACTION ITEMS:\n")
            f.write("-" * 70 + "\n")
            if result['action_items']:
                for i, item in enumerate(result['action_items'], 1):
                    owner = item['owner'] if item['owner'] else 'Unassigned'
                    deadline = item['deadline'] if item['deadline'] else 'No deadline'
                    f.write(f"\n{i}. {item['task']}\n")
                    f.write(f"   Owner: {owner}\n")
                    f.write(f"   Deadline: {deadline}\n")
            else:
                f.write("(No action items detected)\n")
            f.write("\n")

            # Add statistics
            f.write("=" * 70 + "\n")
            f.write("STATISTICS:\n")
            f.write("-" * 70 + "\n")
            f.write(f"Key Points: {len(result['key_points'])}\n")
            f.write(f"Decisions: {len(result['decisions'])}\n")
            f.write(f"Action Items: {len(result['action_items'])}\n")

            unassigned = sum(1 for item in result['action_items'] if not item['owner'])
            no_deadline = sum(1 for item in result['action_items'] if not item['deadline'])

            f.write(f"Unassigned Actions: {unassigned}\n")
            f.write(f"Missing Deadlines: {no_deadline}\n")
            f.write("=" * 70 + "\n")

        print(f"✓ Text Report: {txt_filename}")
        print()

        # Create CSV for action items (if any)
        if result['action_items']:
            csv_filename = f'action_items_{timestamp}.csv'
            with open(csv_filename, 'w', encoding='utf-8') as f:
                f.write("ID,Task,Owner,Deadline,Status\n")
                for i, item in enumerate(result['action_items'], 1):
                    task = item['task'].replace('"', '""')  # Escape quotes
                    owner = item['owner'] if item['owner'] else 'Unassigned'
                    deadline = item['deadline'] if item['deadline'] else 'No deadline'
                    status = 'Pending'
                    f.write(f'{i},"{task}","{owner}","{deadline}","{status}"\n')

            print(f"✓ Action Items CSV: {csv_filename}")
            print()

        print("=" * 70)
        print("📥 TO DOWNLOAD FILES:")
        print("=" * 70)
        print("1. Click the 📁 FOLDER icon in the left sidebar")
        print("2. Find the files listed above")
        print("3. Click ⋮ (three dots) next to each filename")
        print("4. Select 'Download'")
        print()
        print("FILES GENERATED:")
        print(f"  📄 {json_filename} - Machine-readable JSON")
        print(f"  📄 {txt_filename} - Human-readable report")
        if result['action_items']:
            print(f"  📄 {csv_filename} - Action items spreadsheet")
        print()
        print("=" * 70)

    except NameError:
        print("⚠️  Please run Cell 9 first to process the transcript!")
    except Exception as e:
        print(f"❌ Error saving files: {e}")

💾 OUTPUTS SAVED SUCCESSFULLY

✓ JSON File: meeting_analysis_20260103_084248.json
✓ Text Report: meeting_analysis_20260103_084248.txt

✓ Action Items CSV: action_items_20260103_084248.csv

📥 TO DOWNLOAD FILES:
1. Click the 📁 FOLDER icon in the left sidebar
2. Find the files listed above
3. Click ⋮ (three dots) next to each filename
4. Select 'Download'

FILES GENERATED:
  📄 meeting_analysis_20260103_084248.json - Machine-readable JSON
  📄 meeting_analysis_20260103_084248.txt - Human-readable report
  📄 action_items_20260103_084248.csv - Action items spreadsheet

